In [12]:
import ee
import rasterio
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# Initialize Earth Engine with your project ID
ee.Initialize(project='yield-project-2025')

# Define area of interest (replace with your coordinates)
roi = ee.Geometry.Rectangle([-122.4194, 37.7749, -122.4184, 37.7759])  # Example: San Francisco area

# Function to mask clouds in Sentinel-2 Harmonized images (from documentation)
def mask_s2_clouds(image):
    """Masks clouds in a Sentinel-2 Harmonized image using the QA60 band."""
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    
    # Both flags should be set to zero, indicating clear conditions.
    mask = (
        qa.bitwiseAnd(cloud_bit_mask)
        .eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )
    
    return image.updateMask(mask).divide(10000)  # Scale TOA reflectance by 10,000

In [14]:
# Get Sentinel-2 Harmonized imagery
collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate('2025-01-01', '2025-02-27') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .sort('CLOUDY_PIXEL_PERCENTAGE', False) \
    .map(mask_s2_clouds)  # Apply cloud masking
image = collection.first()
image = image.select(['B4', 'B8'])  # Red (B4), NIR (B8)

# Calculate NDVI (Normalized Difference Vegetation Index)
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Export to Google Drive
task = ee.batch.Export.image.toDrive(
    image=ndvi,
    description='ndvi_explore_harmonized',  # Updated description for clarity
    folder='yield_project',
    scale=10,  # Sentinel-2 resolution
    region=roi,
    maxPixels=1e13  # Adjust if quota issues occur
)
task.start()
print("Export task started. Check Google Drive for results.")

Export task started. Check Google Drive for results.


In [17]:
# Load local raster for visualization (after downloading from Google Drive)
with rasterio.open('data/processed/ndvi_explore_harmonized.tif') as src:  # Updated file name
    ndvi_data = src.read(1)

# Visualize NDVI
plt.figure(figsize=(10, 8))
plt.imshow(ndvi_data, cmap='RdYlGn')
plt.colorbar(label='NDVI')
plt.title('NDVI Vegetation Index (Sentinel-2 Harmonized)')
plt.show()

RasterioIOError: data/processed/ndvi_explore_harmonized.tif: No such file or directory